In [1]:
from PIL import Image
import os
import imageio
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, LeakyReLU
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
slum_source = 'Mumbai data/preproccesed data/slum/tiles/'
baseline_source = 'Mumbai data/preproccesed data/baseline/tiles/'

In [4]:
slums = []
for name in os.listdir(slum_source):
    im = imageio.imread(slum_source + '/' + name)
    im = im[:,:,:3]
    slums.append(im)

baseline = []
for name in os.listdir(baseline_source):
    im = imageio.imread(baseline_source + '/' + name)
    im = im[:,:,:3]
    baseline.append(im)

In [5]:
slums = np.array(slums)
baseline = np.array(baseline)


In [6]:
baseline = baseline[:slums.shape[0]]

In [7]:
slums.shape, baseline.shape

((1600, 80, 160, 3), (1600, 80, 160, 3))

In [8]:
labels = np.concatenate((np.zeros(len(slums)),np.ones(len(baseline))))

In [9]:
data = np.concatenate((slums, baseline))

In [10]:
X = data
y = labels
y = to_categorical(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [12]:
adam = optimizers.Adam(lr = .001)

In [30]:
pd.DataFrame(y_test)[0].value_counts(normalize=True)

0.0    0.5
1.0    0.5
Name: 0, dtype: float64

In [14]:
mblnet = MobileNetV2(include_top = True, weights = None, input_shape = (X.shape[1:]), classes = 2)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [31]:
mblnet.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
mblnet.fit(X_train, y_train, epochs=1, validation_data=(X_train, y_train))

Train on 2400 samples, validate on 2400 samples
2400/2400 [==============================] - 190s 79ms/sample - loss: 0.0104 - acc: 0.9975 - val_loss: 0.8365 - val_acc: 0.5000


In [25]:
predictions = mblnet.predict(X_train)

In [34]:
mblnet.evaluate(X_train, y_train)

2400/2400 [==============================] - 29s 12ms/sample - loss: 0.8365 - acc: 0.5000


[0.8365183003743489, 0.5]

In [27]:
for i in predictions:
    print(i)

[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7

[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.707

[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884

[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511  0.29204884]
[0.7079511 0.2920488]
[0.7079511 0.2920488]
[0.7

In [24]:
mblnet.save('stored_image_classifier')